In [37]:
from utils.BackTest import Engine, Strategy
import pandas as pd

In [38]:
df = pd.read_csv(r'../data/processed/combined_df_with_volatility.csv')
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [39]:
df

,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,...,H-L,H-PC,L-PC,TR,ATR,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category
0,2024-03-23 18:00:00+00:00,-0.000662,232.686671,0.000000,65652.70,65681.72,65270.01,65373.88,56859,25413,...,411.71,29.02,382.69,411.71,658.445000,-0.425594,0.005914,0.006518,0.001199,Normal
1,2024-03-23 19:00:00+00:00,-0.000241,243.310915,0.000000,65373.88,65485.15,64786.25,64872.90,73300,32361,...,698.90,111.27,587.63,698.90,701.053333,-0.769282,0.005821,0.006418,0.001168,Normal
2,2024-03-23 20:00:00+00:00,-0.000083,263.874155,0.000000,64872.90,65118.20,64672.93,65009.78,55912,27474,...,445.27,245.30,199.97,445.27,604.058333,0.210775,0.006339,0.006296,0.001028,Normal
3,2024-03-23 21:00:00+00:00,-0.000027,265.004313,0.000000,65009.78,65199.97,64780.00,64944.69,45367,21035,...,419.97,190.19,229.78,419.97,571.245000,-0.100174,0.005951,0.006184,0.000926,Normal
4,2024-03-23 22:00:00+00:00,-0.000008,248.893742,0.000000,64944.69,65050.72,64744.55,64747.86,46105,21611,...,306.17,106.03,200.14,306.17,468.430000,-0.303533,0.005584,0.006049,0.000790,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,22248,...,150.00,109.25,40.75,150.00,298.715000,0.007238,0.003691,0.003776,0.000097,Normal
7406,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,30294,...,358.05,223.01,135.04,358.05,325.368333,0.030520,0.003626,0.003759,0.000091,Low
7407,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,41647,...,586.40,0.01,586.39,586.40,326.935000,-0.381923,0.003572,0.003744,0.000097,Low
7408,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,26960,...,222.90,92.87,130.03,222.90,277.221667,0.081454,0.003957,0.003754,0.000108,High


In [40]:
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

In [41]:
df['typical_price'] = (df['HIGH'] + df['LOW'] + df['CLOSE']) / 3
#df['typical_price'] = df['typical_price'].shift(1) Don't shift as we make decision at the end of each bar, so we have the latest information

In [42]:
#Daily VWAP (UTC-based)
#Reset VWAP every UTC day. This is the most common standard for crypto backtests and analysis.
df['DATE'] = df.index.date  
df['VWAP'] = df.groupby('DATE').apply(
    lambda g: (g['typical_price'] * g['VOLUME']).cumsum() / g['VOLUME'].cumsum()
).reset_index(level=0, drop=True)


# #Rolling VWAP (24-hour window)
# rolling_window = 24  # if hourly bars
# vwap_numerator = (df['typical_price'] * df['VOLUME']).rolling(rolling_window).sum()
# vwap_denominator = df['VOLUME'].rolling(rolling_window).sum()
# df['VWAP'] = vwap_numerator / vwap_denominator

C:\Users\leoqi\AppData\Local\Temp\ipykernel_87224\68637714.py:4: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [43]:
#VWAP strategy
def determine_signal(df):
    """
    Determine the trading signal based on the strategy's criteria.
    """
    if df['CLOSE'] > df['VWAP']:
        return 1
    elif df['CLOSE'] < df['VWAP']:
        return 0
    else:
        return None


In [44]:
df['VWAP_signal'] = df.apply(determine_signal, axis=1)

df['VWAP_signal'] = df['VWAP_signal'].fillna(0)

In [45]:
df_filtered = df.rename(columns={'OPEN':'Open',
                                 'CLOSE':'Close',
                                 'HIGH':'High',
                                 'LOW':'Low'})

In [46]:
class VWAP_Strategy(Strategy):
    def on_bar(self):
        if self.data.loc[self.current_idx]['VWAP_signal'] == 1:
            self.buy(
                "btc",
               size = round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
#                size = round((0.10 * self.cash)/ self.close(), 8)
            )

In [47]:
class BB_RSI_Strategy(Strategy):
    def on_bar(self):
        # Get the current BB_RSI signal
        bb_rsi_signal = self.data.loc[self.current_idx]['BB_RSI_Signal']
        
        # Logic to decide on trading action
        # Buy condition: When VWAP signal is 1 and BB_RSI Signal is 1 (oversold + below lower BB + RSI < 30)
        if bb_rsi_signal == 1:
            self.buy(
                "btc",
                size=round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
            )

In [48]:
class EMA_Strategy(Strategy):
    def on_bar(self):
        # Get the current BB_RSI signal
        EMA_Signal = self.data.loc[self.current_idx]['EMA_Signal']
        
        # Logic to decide on trading action
        # Buy condition: When VWAP signal is 1 and BB_RSI Signal is 1 (oversold + below lower BB + RSI < 30)
        if EMA_Signal == 1:
            self.buy(
                "btc",
                size=round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
            )

In [49]:
class VWAP_BB_RSI_Strategy(Strategy):
    def on_bar(self):
        # Get the current VWAP signal
        vwap_signal = self.data.loc[self.current_idx]['VWAP_signal']
        
        # Get the current BB_RSI signal
        bb_rsi_signal = self.data.loc[self.current_idx]['BB_RSI_Signal']
        
        # Logic to decide on trading action
        # Buy condition: When VWAP signal is 1 and BB_RSI Signal is 1 (oversold + below lower BB + RSI < 30)
        if vwap_signal == 1 and bb_rsi_signal == 1:
            self.buy(
                "btc",
                size=round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
            )

In [50]:
validate_frame = df_filtered[df_filtered.index >= pd.to_datetime('2025-01-01').tz_localize('UTC')]

In [51]:
validate_frame.isna().sum()

reddit_smoothed_sentiment_weibull    0
news_smoothed_sentiment_weibull      0
tele_smoothed_sentiment_weibull      0
Open                                 0
High                                 0
Low                                  0
Close                                0
TOTAL_TRADES                         0
TOTAL_TRADES_BUY                     0
TOTAL_TRADES_SELL                    0
VOLUME                               0
VOLUME_BUY                           0
VOLUME_SELL                          0
return                               0
log_return                           0
hourly_volatility                    0
EMA_8                                0
EMA_13                               0
EMA_21                               0
EMA_Signal                           0
EMA_short                            0
EMA_long                             0
MACD                                 0
Signal_Line                          0
MACD_Signal                          0
MACD_Hist                

In [52]:
validate_frame['RSI_14'].tail(50)

datetime
2025-01-24 10:00:00+00:00    68.905952
2025-01-24 11:00:00+00:00    71.528204
2025-01-24 12:00:00+00:00    66.664657
2025-01-24 13:00:00+00:00    62.116387
2025-01-24 14:00:00+00:00    72.396746
2025-01-24 15:00:00+00:00    75.998170
2025-01-24 16:00:00+00:00    76.061432
2025-01-24 17:00:00+00:00    75.757922
2025-01-24 18:00:00+00:00    69.389211
2025-01-24 19:00:00+00:00    56.193715
2025-01-24 20:00:00+00:00    48.686302
2025-01-24 21:00:00+00:00    50.743897
2025-01-24 22:00:00+00:00    42.939010
2025-01-24 23:00:00+00:00    47.160176
2025-01-25 00:00:00+00:00    40.113973
2025-01-25 01:00:00+00:00    43.635156
2025-01-25 02:00:00+00:00    41.095921
2025-01-25 03:00:00+00:00    50.744351
2025-01-25 04:00:00+00:00    40.052230
2025-01-25 05:00:00+00:00    37.029339
2025-01-25 06:00:00+00:00    38.061099
2025-01-25 07:00:00+00:00    28.715328
2025-01-25 08:00:00+00:00    29.236219
2025-01-25 09:00:00+00:00    33.556153
2025-01-25 10:00:00+00:00    45.455762
2025-01-25 11:00

In [53]:
validate_frame[['BB_RSI_Signal', 'VWAP_signal']].tail(50)

,BB_RSI_Signal,VWAP_signal
datetime,,
2025-01-24 10:00:00+00:00,0,1
2025-01-24 11:00:00+00:00,0,1
2025-01-24 12:00:00+00:00,0,1
2025-01-24 13:00:00+00:00,0,1
2025-01-24 14:00:00+00:00,0,1
2025-01-24 15:00:00+00:00,0,1
2025-01-24 16:00:00+00:00,0,1
2025-01-24 17:00:00+00:00,0,1
2025-01-24 18:00:00+00:00,0,1


In [54]:
e = Engine(initial_cash = 100_000,asset_type='cryptocurrencies')
e.add_data(validate_frame)
e.add_strategy(EMA_Strategy()) # Pass both df_filtered and strategy_dict
stats = e.run()

  0%|          | 0/612 [00:00<?, ?it/s]

 28%|██▊       | 171/612 [00:00<00:00, 858.83it/s]

2025-01-01 15:00:00+00:00 New position opened: <Position: btc size: 0.10618429 entry: 94175.89>,transaction fee: 10.0000000147681
2025-01-01 16:00:00+00:00 New position opened: <Position: btc size: 0.11651368 entry: 94432.94>,transaction fee: 11.0027293526192
2025-01-01 17:00:00+00:00 Stop Loss Filled. Limit: 93886.06583333333 / Low: 93760.93
2025-01-01 17:00:00+00:00 Position closed for ticker btc at 93886.06583333333, transaction fee: 9.96922524140576
2025-01-01 17:00:00+00:00 Stop Loss Filled. Limit: 94143.11583333333 / Low: 93760.93
2025-01-01 17:00:00+00:00 Position closed for ticker btc at 94143.11583333333, transaction fee: 10.968960872407932
2025-01-01 17:00:00+00:00 New position opened: <Position: btc size: 0.12855046 entry: 94090.49>,transaction fee: 12.095375771125402
2025-01-01 18:00:00+00:00 New position opened: <Position: btc size: 0.11907547 entry: 94143.85>,transaction fee: 11.210223186359501
2025-01-01 19:00:00+00:00 New position opened: <Position: btc size: 0.1307621 

 70%|███████   | 430/612 [00:00<00:00, 863.02it/s] 

2025-01-12 03:00:00+00:00 Stop Loss Filled. Limit: 94426.17666666667 / Low: 94418.56
2025-01-12 03:00:00+00:00 Position closed for ticker btc at 94426.17666666667, transaction fee: 11.0942551948581
2025-01-12 03:00:00+00:00 New position opened: <Position: btc size: 0.12918988 entry: 94625.25>,transaction fee: 12.224624692470002
2025-01-12 04:00:00+00:00 New position opened: <Position: btc size: 0.13036292 entry: 94622.4>,transaction fee: 12.335252361408
2025-01-12 05:00:00+00:00 Stop Loss Filled. Limit: 94512.15583333334 / Low: 94500.0
2025-01-12 05:00:00+00:00 Position closed for ticker btc at 94512.15583333334, transaction fee: 12.210014070649635
2025-01-12 05:00:00+00:00 Stop Loss Filled. Limit: 94509.30583333333 / Low: 94500.0
2025-01-12 05:00:00+00:00 Position closed for ticker btc at 94509.30583333333, transaction fee: 12.320509075606365
2025-01-12 05:00:00+00:00 New position opened: <Position: btc size: 0.14343766 entry: 94587.99>,transaction fee: 13.5674799497034
2025-01-12 06:

100%|██████████| 612/612 [00:00<00:00, 917.83it/s]

2025-01-18 22:00:00+00:00 New position opened: <Position: btc size: 0.14368508 entry: 103889.29>,transaction fee: 14.9273409447932
2025-01-18 23:00:00+00:00 Take Profit Filled. Limit: 104139.365 / High: 104750.0
2025-01-18 23:00:00+00:00 Position closed for ticker btc at 104139.365, transaction fee: 11.191789865962752
2025-01-18 23:00:00+00:00 Take Profit Filled. Limit: 104711.865 / High: 104750.0
2025-01-18 23:00:00+00:00 Position closed for ticker btc at 104711.865, transaction fee: 11.2106386540197
2025-01-18 23:00:00+00:00 New position opened: <Position: btc size: 0.15768255 entry: 104291.3>,transaction fee: 16.444918126815004
2025-01-19 00:00:00+00:00 New position opened: <Position: btc size: 0.15175474 entry: 104556.23>,transaction fee: 15.8669034990302
2025-01-19 01:00:00+00:00 Take Profit Filled. Limit: 104609.10749999998 / High: 104684.22
2025-01-19 01:00:00+00:00 Position closed for ticker btc at 104609.10749999998, transaction fee: 15.030767979866098
2025-01-19 01:00:00+00:0

In [55]:
e.plot(show_signals= False)